In [ ]:
:e Arrows
:e QuasiQuotes
import Control.Arrow
import Control.Funflow
import Control.Funflow.Exec.Simple (withSimpleLocalRunner)
import Data.Default
import Path

-- | Flow which asks for the user's name, then their favourite number.
--   The favourite number will be cached, such that if the user comes back,
--   they will not be asked again.
favouriteNumber :: SimpleFlow () (String, String)
favouriteNumber = proc () -> do
    name <- stepIO (\_ -> putStrLn "Hi, what's your name?" >> getLine) -< ()
    -- We enable caching for this step. The default cacher uses 'Store' instances
    -- to provide serialisation/deserialisation. The ident is used to ensure that
    -- multiple different steps with the same input do not resolve to the same
    -- cache item.
    fnum <- stepIO' (def { cache = defaultCacherWithIdent 172652 })
                  (\_ -> putStrLn "What's your favourite number?" >> getLine) -< name
    returnA -< (name, fnum)

-- | Runs the 'favourite number' flow in IO.

-- Use /tmp/funflow as the path to our content store.
-- | Runs the 'favourite number' flow in IO.
runFavouriteNumber :: IO ()
runFavouriteNumber = do
  -- Use /tmp/funflow as the path to our content store.
  res <- withSimpleLocalRunner [absdir|/tmp/funflow|] $ \run ->
    -- 'run' takes the flow and the initial input. Since this flow has an input type of '()',
    -- this is what we provide.
    run favouriteNumber ()
  case res of
    Left err -> putStrLn $ "Something went wrong: " ++ show err
    Right (name, num) -> putStrLn $ "Hi, " ++ name ++ ", your favourite number is " ++ num
    
runFavouriteNumber

Hi, what's your name?
[2019-01-24 16:42:01][FFExecutorD.executeLoop][Info][matto-dell][4717][ThreadId 199][funflow-1.4.0-KLdImYfU5mYAAbPXU2bOEB:Control.Funflow.External.Executor src/Control/Funflow/External/Executor.hs:181:9] Initialising connection to coordinator.
Message: PublishDisplayData {header = MessageHeader {mhIdentifiers = ["aff67426-81c2-49f3-a75b-dd4793b09a7d"], mhParentHeader = Just (MessageHeader {mhIdentifiers = ["aff67426-81c2-49f3-a75b-dd4793b09a7d"], mhParentHeader = Nothing, mhMetadata = Metadata (fromList [("deletedCells",Array []),("cellId",String "fe97d40f-3be3-44c5-8053-56358fa62843")]), mhMessageId = UUID {uuidToString = "e38a1a27-bbf5-4471-a739-9b83d42d49d0"}, mhSessionId = UUID {uuidToString = "aff67426-81c2-49f3-a75b-dd4793b09a7d"}, mhUsername = "", mhMsgType = ExecuteRequestMessage}), mhMetadata = Metadata (fromList []), mhMessageId = UUID {uuidToString = "c793b500-3f54-4a72-8f87-937c892d63ed"}, mhSessionId = UUID {uuidToString = "aff67426-81c2-49f3-a75b-dd4